下記のリンクをクリックするとGoogle Colabで実行することが出来ます  
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](http://colab.research.google.com/github/2Nike2/LangChainPractice/blob/main/00_03_use_retriever.ipynb)

### (事前準備: OpenAI APIキーの設定)
OpenAI APIを使う為のAPIキーを設定します  
このAPIキーについては、OpenAIのサイトで取得することが出来ます  
https://platform.openai.com/api-keys  
APIキーについては公開しないように注意してください  

In [ ]:
import os

# ここにあなたのOpenAIのAPIキーを入力してください
openai_api_key = 'yourapikey'

# 環境変数にAPIキーがまだ設定されていないならばAPIキーを設定
if os.getenv('OPENAI_API_KEY') is None:
    os.environ['OPENAI_API_KEY'] = openai_api_key


## Web文書をソースとしたリトリーバを使ったRAG
検索した文書を指示、質問と共にコンテキストとしてLLMに渡すことで正確な回答を得る手法RAG(Retrieval Augmented Generation:検索強化生成)の動作を確認します

### ライブラリのインストール
LangChainのライブラリをインストールします  
またWeb文書を取得するためのライブラリであるBeautifulSoup  
及びベクトルデータベースのfaiss(CPU版)もインストールします  


In [ ]:
!pip install langchain==0.1.4
!pip install langchain-openai==0.0.5
!pip install beautifulsoup4==4.12.3
!pip install faiss-cpu==1.7.4


### モデルの初期化
OpenAI APIを使う為のモデルを初期化します

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)


### Web文書の取得
WebBaseLoaderを使ってWeb文書を取得します

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://www.aozora.gr.jp/cards/000081/files/43754_17659.html") # 青空文庫 「注文の多い料理店」(宮沢賢治)

docs = loader.load()


### 埋め込みモデル
埋め込みモデルを初期化します
これは今まで使ってきた文章に対して文章を返すモデルと違い、文章に対して埋め込みベクトル(数百~数千個の数値のリスト)を返すモデルです  
個の埋め込みベクトルというのは文章の意味を数値に要約したものと考えられ、複数の埋め込みベクトルの距離や角度を計算することによって文章の意味の近さを捉えられます  
これを利用して指示や質問に対して意味が近かったり関連性が高い文章を探し出すことが出来ます

In [ ]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()


### ベクトルデータベースの用意
上記の埋め込みベクトルを格納して高速に検索するためのベクトルデータベースを用意します  
ベクトルデータベースに登録する文章は全体をそのまま入れるのではなく、何らかの単位(字数、章、ページ区切り等)で分割してからベクトル化して保存することになります

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

# text_splitter = RecursiveCharacterTextSplitter()
text_splitter = CharacterTextSplitter(separator='\n', chunk_size=800, chunk_overlap=100)
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)


### リトリーバの作成

In [ ]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever(search_kwargs={'k': 3})


## 会話履歴リトリーバ
会話履歴を考慮したリトリーバを作ります

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt_retriever_query = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name='chat_history'),
    ('user', '{input}'),
    ('user', '上記の会話について、関係する情報を取得できる様な検索クエリを作成して下さい。'),
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt_retriever_query)


#### 会話履歴リトリーバの確認

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    HumanMessage(content='ガラスのツボに入っていたものは？'), 
    AIMessage(content='壺のなかのものは牛乳のクリームでした。')
    ]
question = 'それをどの様にして使いましたか？'

retriever_chain.invoke({
    "chat_history": chat_history,
    "input": question
})


In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.messages import SystemMessage

prompt_to_question = ChatPromptTemplate.from_messages([
    ('system', '以下の文脈に基づいてユーザの質問に答えて下さい。\n\n{context}'),
    MessagesPlaceholder(variable_name="chat_history"),
    ('user', '{input}')
])

# prompt_to_question = ChatPromptTemplate.from_messages([
#     SystemMessage(content='以下の文脈に基づいてユーザの質問に答えて下さい。\n\n{context}'),
#     MessagesPlaceholder(variable_name="chat_history"),
#     HumanMessage(content='{input}')
# ])

document_chain = create_stuff_documents_chain(llm, prompt_to_question)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": question
})


#### (参考) 文脈を与えずに同じ質問をした場合

In [ ]:
retrieval_chain.invoke({
    "chat_history": [],
    "input": question
})